 # 1.. lets read data..

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv(r'hotel_bookings.csv')

In [ ]:
type(df)

In [ ]:
df.head(3)

# 2.. lets perform data cleaning..

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['agent','company'],axis=1,inplace=True)

In [ ]:
df['country'].value_counts().index[0]

In [ ]:
df['country'].fillna(df['country'].value_counts().index[0],inplace=True)

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
### seems to have some dirtiness in data as Adults,babies & children cant be zero at a same time ..

### bcz if 3 entities are 0 ,then how can a booking be possible ??

In [ ]:
### Visualise Entire Dataframe where adult,children & babies are 0

filter1=(df['children']==0) & (df['adults']==0) & (df['babies']==0)

In [ ]:
df[filter1]

In [ ]:
data=df[~filter1]

In [ ]:
data.shape

In [ ]:
df.shape

In [ ]:
df[df['children']==0]

# 3.. Where do the guests come from ?

In [ ]:
## Lets perform Spatial Analysis

In [ ]:
data['is_canceled'].unique()

In [ ]:
data[data['is_canceled']==0]['country'].value_counts()/75011

In [ ]:
len(data[data['is_canceled']==0])

In [ ]:
country_wise_data=data[data['is_canceled']==0]['country'].value_counts().reset_index()
country_wise_data.columns=['country','no_of_guests']
country_wise_data

In [ ]:
##!pip install plotly

In [ ]:
##!pip install chart_studio

In [ ]:
import plotly
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs ,init_notebook_mode ,plot ,iplot
init_notebook_mode(connected=True)

In [ ]:
import plotly.express as px

In [ ]:
map_guest=px.choropleth(country_wise_data,
             locations=country_wise_data['country'],
             color=country_wise_data['no_of_guests'],
              hover_name=country_wise_data['country'],
              title='home country of guests'
             )

In [ ]:
map_guest.show()

# 4.. How much do guests pay for a room per night ?

In [ ]:
data2=data[data['is_canceled']==0]

In [ ]:
data2.columns

In [ ]:
# seaborn boxplot:

plt.figure(figsize=(12,8))
sns.boxplot(x='reserved_room_type',y='adr' ,hue='hotel',data=data2)

plt.title('Price of room types per night and person')
plt.xlabel('room types')
plt.ylabel('price( EUR)')

# 5.. Which are the most busy month ?

In [ ]:
data['hotel'].unique()

In [ ]:
data_resort=data[(data['hotel']=='Hilton Hotel') & (data['is_canceled']==0)]
data_city = data[(data['hotel']=='City Hotel') & (data['is_canceled']==0)]

In [ ]:
data_resort.head(3)

In [ ]:
rush_resort=data_resort['arrival_date_month'].value_counts().reset_index()
rush_resort.columns=['month','no_of_guests']
rush_resort

In [ ]:
rush_city=data_city['arrival_date_month'].value_counts().reset_index()
rush_city.columns=['month','no_of_guests']
rush_city

In [ ]:
final_rush=rush_resort.merge(rush_city,on='month')

In [ ]:
final_rush.columns=['month','no_of_guests_in_resort','no_of_guests_city']

In [ ]:
final_rush

In [ ]:
!pip install sorted-months-weekdays

## Dependency package needs to be installed
!pip install sort_dataframeby_monthorweek

In [ ]:
import sort_dataframeby_monthorweek as sd

In [ ]:
final_rush=sd.Sort_Dataframeby_Month(final_rush,'month')

In [ ]:
final_rush.columns

In [ ]:
px.line(data_frame=final_rush,x='month',y=['no_of_guests_in_resort', 'no_of_guests_city'])

# 6.. which month has highest adr ?

In [ ]:
data=sd.Sort_Dataframeby_Month(data,'arrival_date_month')

In [ ]:
sns.barplot(x='arrival_date_month',y='adr',data=data ,hue='is_canceled')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='arrival_date_month',y='adr',data=data ,hue='is_canceled')
plt.xticks(rotation='vertical')


plt.ylim(0,800)
plt.show()

# 7.. Lets analyse whether bookings were made only for weekdays or for weekends or for both ??

In [ ]:
data.columns

In [ ]:
### Lets create a relationship table..
pd.crosstab(index=data['stays_in_weekend_nights'],columns=data['stays_in_week_nights'])

In [ ]:
## lets define our own function :

def week_function(row):
    feature1='stays_in_weekend_nights'
    feature2='stays_in_week_nights'
    
    if row[feature2]==0 and row[feature1] >0 :
        return 'stay_just_weekend'
    
    elif row[feature2]>0 and row[feature1] ==0 :
        return 'stay_just_weekdays'
    
    elif row[feature2]>0 and row[feature1] >0 :
        return 'stay_both_weekdays_weekends'
    
    else:
        return 'undefined_data'

In [ ]:
data2['weekend_or_weekday']=data2.apply(week_function,axis=1)

In [ ]:
data2.head(2)

In [ ]:
data2['weekend_or_weekday'].value_counts()

In [ ]:
type(sd)

In [ ]:
data2=sd.Sort_Dataframeby_Month(data2,'arrival_date_month')

In [ ]:
data2.groupby(['arrival_date_month','weekend_or_weekday']).size()

In [ ]:
group_data=data2.groupby(['arrival_date_month','weekend_or_weekday']).size().unstack().reset_index()

In [ ]:
sorted_data=sd.Sort_Dataframeby_Month(group_data,'arrival_date_month')

In [ ]:
sorted_data.set_index('arrival_date_month',inplace=True)

In [ ]:
sorted_data

In [ ]:
sorted_data.plot(kind='bar',stacked=True,figsize=(15,10))

# 8.. How to create some more features ..

In [ ]:
data2.columns

In [ ]:
def family(row):
    if (row['adults']>0) &  (row['children']>0 or row['babies']>0) :
        return 1
    else:
        return 0

In [ ]:
data['is_family']=data.apply(family,axis=1)

In [ ]:
data['total_customer'] = data['adults'] + data['babies'] + data['children']

In [ ]:
data['total_nights']=data['stays_in_week_nights'] + data['stays_in_weekend_nights']

In [ ]:
data.head(3)

In [ ]:
data.columns

In [ ]:
data['deposit_type'].unique()

In [ ]:
dict1={'No Deposit':0, 'Non Refund':1, 'Refundable': 0}

In [ ]:
data['deposit_given']=data['deposit_type'].map(dict1)

In [ ]:
data.columns

In [ ]:
data.drop(columns=['adults', 'children', 'babies', 'deposit_type'],axis=1,inplace=True)

In [ ]:
data.columns

# 9.. how to apply Feature encoding on data 

In [ ]:
data.head(6)

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
cate_features=[col for col in data.columns if data[col].dtype=='object']

In [ ]:
num_features=[col for col in data.columns if data[col].dtype!='object']

In [ ]:
num_features

In [ ]:
cate_features

In [ ]:
data_cat=data[cate_features]

In [ ]:
data.groupby(['hotel'])['is_canceled'].mean().to_dict()

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
data_cat['cancellation']=data['is_canceled']

In [ ]:
data_cat.head()

In [ ]:
cols=data_cat.columns

In [ ]:
cols=cols[0:-1]

In [ ]:
cols

In [ ]:
### Perform Mean Encoding Technique 

for col in cols:
    dict2=data_cat.groupby([col])['cancellation'].mean().to_dict()
    data_cat[col]=data_cat[col].map(dict2)

In [ ]:
data_cat.head(3)

# 10.. Handle Outliers 

In [ ]:
data[num_features]

In [ ]:
dataframe=pd.concat([data_cat,data[num_features]],axis=1)

In [ ]:
dataframe.columns

In [ ]:
dataframe.drop(['cancellation'],axis=1,inplace=True)

In [ ]:
dataframe.head(3)

In [ ]:
sns.distplot(dataframe['lead_time'])

In [ ]:
def handle_outlier(col):
    dataframe[col]=np.log1p(dataframe[col])

In [ ]:
handle_outlier('lead_time')

In [ ]:
sns.distplot(dataframe['lead_time'])

In [ ]:
## adr

In [ ]:
sns.distplot(dataframe['adr'])

In [ ]:
dataframe[dataframe['adr']<0]

In [ ]:
handle_outlier('adr')

In [ ]:
dataframe['adr'].isnull().sum()

In [ ]:
### now why this missing value , as we have already deal with the missing values..'
### bcz we have negative value in 'adr' feature as '-6.38'  ,& if we apply ln(1+x) , we will get 'nan'
## bcz log wont take negative values..

In [ ]:
sns.distplot(dataframe['adr'].dropna())

# 11.. Select important Features using Co-relation & univariate analysis..

In [ ]:
sns.FacetGrid(data,hue='is_canceled',xlim=(0,500)).map(sns.kdeplot,'lead_time',shade=True).add_legend()

In [ ]:
corr=dataframe.corr()

In [ ]:
corr

In [ ]:
corr['is_canceled'].sort_values(ascending=False)

In [ ]:
corr['is_canceled'].sort_values(ascending=False).index

In [ ]:
features_to_drop=['reservation_status', 'reservation_status_date','arrival_date_year',
       'arrival_date_week_number', 'stays_in_weekend_nights',
       'arrival_date_day_of_month']

In [ ]:
dataframe.drop(features_to_drop,axis=1,inplace=True)

In [ ]:
dataframe.shape

# 12.. How to find Important features for model building..

In [ ]:
dataframe.head(2)

In [ ]:
dataframe.isnull().sum()

In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
## separate dependent & independent features

In [ ]:
x=dataframe.drop('is_canceled',axis=1)

In [ ]:
y=dataframe['is_canceled']

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [ ]:
##Lasso(alpha=0.005)
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.


In [ ]:
feature_sel_model=SelectFromModel(Lasso(alpha=0.005))

In [ ]:
feature_sel_model.fit(x,y)

In [ ]:
feature_sel_model.get_support()

In [ ]:
cols=x.columns

In [ ]:
cols

In [ ]:
# let's print the number of selected features

selected_feature=cols[feature_sel_model.get_support()]

In [ ]:
selected_feature

In [ ]:
x=x[selected_feature]

In [ ]:
y

# 13.. Lets build ML model..

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.25)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
logreg=LogisticRegression()

In [ ]:
logreg.fit(X_train,y_train)

In [ ]:
pred=logreg.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,pred)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,pred)

# 14.. How to cross-validate model..

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
score=cross_val_score(logreg,x,y,cv=10)

In [ ]:
score

In [ ]:
score.mean()

# 15.. playing with multiple algos..

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier



In [ ]:
models=[]

models.append(('LogisticRegression',LogisticRegression()))
models.append(('Naive_bayes',GaussianNB()))
models.append(('Random Forest',RandomForestClassifier()))
models.append(('Decision_tree',DecisionTreeClassifier()))
models.append(('KNN',KNeighborsClassifier()))

In [ ]:
for name,model in models:
    print(name)
    model.fit(X_train,y_train)
    
    predictions=model.predict(X_test)
    
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(predictions,y_test)
    print(cm)
    
    from sklearn.metrics import accuracy_score
    acc=accuracy_score(predictions,y_test)
    print(acc)
    print('\n')